In [17]:
#Perform analysis
from tqdm import tqdm
import os
import numpy as np
import pandas as pd
import concurrent.futures
from nimlab import datasets as nimds
from calvin_utils.palm import whole_brain_permutation_test
from calvin_utils.palm import permute_contrast_matrix


#----------------------------------------------------------------Begin User Input
# Gather information from terminal
n_permutations = 1
matrix_to_test = '/Users/cu135/Dropbox (Partners HealthCare)/memory/functional_networks/response_topology/voxelwise_glm/age_interaction_rios_vtas/t_values_topology/matrix_to_test.csv'
t_matrix = '/Users/cu135/Dropbox (Partners HealthCare)/memory/functional_networks/response_topology/voxelwise_glm/age_interaction_rios_vtas/t_values_topology/t_values.csv'
out_dir = '/Users/cu135/Dropbox (Partners HealthCare)/memory/analyses/test_env' #input("Enter output directory: ")
job_name = '5perms_5_workers'
#----------------------------------------------------------------End User Input

# Import the data
matrix_to_test = pd.read_csv(matrix_to_test,index_col=False)
try:
    matrix_to_test.pop('Patient # CDR, ADAS')
except:
    print('Could not remove Patient # CDR, ADAS column')
t_matrix = pd.read_csv(t_matrix,index_col=False)



In [18]:
# matrix_to_test = matrix_to_test.iloc[:,:5]
display(matrix_to_test)

,stdev_improvement_adascog11,age,93491,93492,93493,93582,93583,93672,93673,93674,...,798109,798110,798190,798191,798192,798193,798281,798282,798283,798373
0,0.314000,62.0,-0.25830,-0.5034,-0.5957,-0.39380,-0.5537,-0.237800,-0.4610,-0.5117,...,0.33620,-0.20780,0.22240,0.19120,-0.008150,-0.273200,0.22380,0.16930,0.007553,0.09420
1,0.014000,77.0,-0.25760,-0.4915,-0.5580,-0.39500,-0.5347,-0.270300,-0.4840,-0.5040,...,0.47580,-0.02924,0.20240,0.16750,-0.000731,-0.217900,0.18930,0.14380,0.000192,0.07935
2,-0.842000,76.0,-0.11380,-0.3809,-0.4958,-0.22680,-0.4307,-0.024780,-0.2964,-0.4111,...,0.30080,-0.27030,0.55760,0.50440,0.285400,0.004300,0.57600,0.50900,0.346200,0.47750
3,-1.855000,65.0,-0.39870,-0.6930,-0.8320,-0.54250,-0.7373,-0.398400,-0.6157,-0.7144,...,-0.13960,-0.56350,0.02411,-0.01985,-0.237200,-0.531000,0.04953,-0.02629,-0.179300,-0.04755
4,0.533000,50.0,-0.89360,-1.2210,-1.4770,-1.21900,-1.3980,-1.264000,-1.4700,-1.5170,...,-1.04700,-1.67800,-0.84330,-0.79350,-0.832000,-1.080000,-0.83350,-0.77800,-0.776000,-0.72000
5,-0.028150,66.0,-0.18000,-0.4377,-0.5522,-0.27400,-0.4783,-0.030530,-0.3179,-0.4521,...,0.07184,-0.44210,0.57030,0.47920,0.222800,-0.080900,0.58400,0.47020,0.278300,0.45070
6,0.435500,64.0,-0.72200,-0.9600,-1.1450,-1.02800,-1.1250,-1.157000,-1.2660,-1.2310,...,-0.86870,-1.44000,-0.94700,-0.86100,-0.821000,-0.961000,-0.89160,-0.77000,-0.665000,-0.62400
7,0.158600,60.0,-0.52640,-0.8096,-1.0420,-0.80400,-0.9507,-0.953000,-1.0440,-1.0620,...,-0.90970,-1.45800,-0.86570,-0.79540,-0.755000,-0.905000,-0.79700,-0.67700,-0.558000,-0.48900
8,0.133200,72.0,-0.14810,-0.4214,-0.5703,-0.21570,-0.4314,0.022660,-0.2576,-0.3984,...,-0.05690,-0.56540,0.77200,0.63600,0.349000,0.028060,0.78100,0.62500,0.396200,0.59470
9,-0.955600,72.0,-0.36600,-0.5986,-0.7460,-0.39920,-0.6094,-0.105800,-0.3923,-0.5854,...,-0.58200,-1.03000,0.87940,0.60700,0.185400,-0.207500,0.87160,0.57130,0.234400,0.56450


In [19]:
permuted_matrix = permute_contrast_matrix(matrix_to_test, voxel_index=2, looped_permutation=True)
display(permuted_matrix)

,vox_0,vox_1,vox_2,vox_3,vox_4,vox_5,vox_6,vox_7,vox_8,vox_9,...,vox_225214,vox_225215,vox_225216,vox_225217,vox_225218,vox_225219,vox_225220,vox_225221,vox_225222,vox_225223
0,-1.265000,67.0,-1.6660,-1.31100,2.8280,1.093000,-1.56900,-0.5860,-0.82760,-1.70300,...,2.547000,2.94700,0.25460,4.41800,1.81800,0.7793,-0.78170,0.06964,1.51800,0.006836
1,1.110000,68.0,-1.3940,0.88040,-1.3350,-1.945000,-1.56250,-2.0250,-1.41300,3.45500,...,-0.272700,0.39870,-0.17250,-1.76100,-0.39870,0.0706,-1.93500,-2.44500,-1.40100,0.160200
2,-0.008835,68.0,0.4382,2.01200,0.8975,0.863300,-1.30700,-0.7817,0.81640,0.17960,...,1.320000,-0.20240,0.69500,-0.24560,-0.19170,-1.3850,0.54830,0.78660,0.87350,1.373000
3,1.582000,77.0,-1.7150,-2.04500,6.2730,-0.076600,-0.42630,-1.0440,-0.90900,1.19800,...,-0.954000,0.01114,-2.21500,1.00200,-2.80300,-3.2010,-1.97900,0.05340,-1.58500,-0.761700
4,1.841000,65.0,-0.5864,2.22700,2.4280,0.245400,3.56000,-0.6313,2.18200,0.38300,...,-0.510000,0.81540,-0.92140,-2.75800,-3.12300,-0.3286,-0.70500,0.13200,0.10693,-0.521000
5,-0.955600,58.0,0.1477,-3.43000,-0.1848,0.757000,-2.61300,-1.4370,-1.99200,-0.75150,...,-3.363000,-0.81300,-2.54700,-0.32470,1.99400,-3.1070,0.25950,-1.38100,1.35900,4.980000
6,0.027040,71.0,0.1417,-0.84670,-1.9940,2.350000,0.56200,2.4000,-0.83700,-1.18300,...,1.470000,1.23100,0.57960,-1.39400,-0.41530,-1.7410,-2.08000,-1.39000,-1.43600,-2.150000
7,-3.428000,69.0,-1.2280,1.56000,4.1300,-3.934000,-0.51800,-1.4980,0.64650,-2.22700,...,-0.618700,-1.25900,0.19910,-1.36100,-0.65040,0.5254,-1.65200,1.03700,1.77500,-0.666500
8,-0.225300,66.0,-1.2750,-0.06836,2.1760,1.173000,-1.61000,-1.1040,-2.59400,-1.85800,...,1.638000,-0.60450,0.52000,0.95100,0.05087,1.0960,-1.74000,1.37500,1.09200,-2.662000
9,1.019000,60.0,-1.1820,1.78800,1.7110,0.120100,-0.54100,1.5910,-1.03700,1.23200,...,-1.463000,-1.98500,3.74800,-0.36840,1.62500,-1.8390,-0.44170,1.13300,0.53760,0.190300


In [ ]:
#Prepare the empiric t values matrix
mni_mask = nimds.get_img("mni_icbm152")
mask_data = mni_mask.get_fdata().flatten()
brain_indices = np.where(mask_data > 0)[0]
t_matrix = t_matrix.to_numpy()[brain_indices, -1]

# Launch Multiprocessing of PALM Analyses
p_matrix = np.zeros_like(t_matrix)
with concurrent.futures.ProcessPoolExecutor(max_workers=n_permutations) as executor:
	#Begin submitting the masked data to the permutor
	results = []
	for i in tqdm(range(n_permutations), desc="Jobs Launched"):
		#This is not optimal, but it is necessary
		#Avoids identical permutations by ensuring pseudo-random permutations all occur from the same core
		permuted_matrix = permute_contrast_matrix(matrix_to_test, voxel_index=2)
  
		#Assign the permuted data to a worker. return the result
		result = executor.submit(whole_brain_permutation_test, permuted_matrix, t_matrix)
		results.append(result)
		
	  	 # Limit number of workers at given time to prevent memory pressure issues
		 # jobs[result] = i #Add job to dict of ongoing jobs
		# if len(jobs) > max_jobs-1: #Check number of ongoing jobs
		#     completed_jobs, _ = concurrent.futures.wait(jobs.keys(), return_when=concurrent.futures.FIRST_COMPLETED)
		#     # Remove the completed job from the dict of ongoing jobs
		#     del jobs[completed_jobs.pop()]
	# progress_bar = tqdm(total=n_permutations, desc="Jobs Finalized")
	# for result in concurrent.futures.as_completed(results):
	# 	#Input the permuted data into the array
	# 	extracted_p_values = result.result()
	# 	p_matrix = p_matrix + extracted_p_values
		
	# 	#Update visualization
	# 	progress_bar.update()
	# progress_bar.close()

# Generate a filename depending on the preceding files
# i = 0
# unsaved = True
# while unsaved:
# 	out_file = os.path.join(out_dir, f"{job_name}_{i}.csv") # Construct the output file path
# 	if os.path.exists(out_file):
# 		i += 1
# 	else:
# 		p_df = pd.DataFrame(p_matrix) # Call the function that generates the vector
# 		# p_df.to_csv(out_file, index=False, header=False) # Save the output to the file
# 		unsaved=False

AttributeError: 'numpy.ndarray' object has no attribute 'to_numpy'

In [11]:
display(permuted_matrix)

,vox_0,vox_1,vox_2,vox_3,vox_4,vox_5,vox_6,vox_7,vox_8,vox_9,...,vox_225214,vox_225215,vox_225216,vox_225217,vox_225218,vox_225219,vox_225220,vox_225221,vox_225222,vox_225223
0,0.533000,67.0,-1.76800,0.00999,1.71600,-0.742000,-2.258000,-0.91100,3.5940,-0.57030,...,-1.195000,-0.39040,1.750000,-0.21240,-1.949000,-2.57800,-1.05500,-0.07855,-0.98050,-1.75100
1,-0.059050,66.0,-0.38840,0.58250,-1.04100,-3.455000,-0.573000,-1.38700,-1.9795,1.72100,...,-1.715000,-0.02678,2.033000,7.40600,-1.578000,0.55300,2.40800,-1.79800,0.46310,-0.72070
2,-2.068000,72.0,-1.24300,-0.10913,5.30500,-2.803000,-2.018000,-1.24400,0.0604,-0.92140,...,-2.229000,-0.98730,1.699000,-0.14860,-1.570000,2.02100,1.03200,-0.79000,-2.26600,-2.90200
3,-0.842000,58.0,0.58900,3.46900,-1.22700,-0.629400,0.229200,-1.53200,-2.6130,-1.36700,...,-2.047000,-1.27300,-1.897000,6.49000,-1.210000,1.17600,0.29300,1.78300,1.83500,0.14940
4,0.649000,72.0,-3.31400,-1.02300,0.74900,1.345000,-2.564000,0.80900,1.0070,0.58250,...,-1.424000,3.41000,2.025000,-0.92330,1.212000,-1.95500,-2.58600,-1.07600,-2.84800,2.13700
5,-0.808000,71.0,1.05200,-0.59960,-0.54640,-0.673300,-2.002000,-1.12500,-0.7173,-2.82400,...,-1.459000,1.06500,-0.327400,-2.97500,-1.911000,1.29800,2.75000,3.44700,-1.12300,1.13100
6,-0.008835,47.0,2.60400,-2.82400,0.68400,-0.760300,-1.661000,-0.69100,-2.1230,0.02162,...,4.004000,-3.15400,-1.003000,-2.20000,-0.911000,-2.79000,-1.82100,-0.92000,1.75900,-0.61430
7,0.435500,68.0,-0.89450,1.59900,-2.07200,-0.878400,-1.371000,2.95000,-3.0940,-0.40900,...,-1.036000,-0.54640,-0.634000,1.75700,0.696000,0.84100,0.11670,-1.67800,0.77340,1.36400
8,-0.028150,79.0,-2.38700,0.04224,-2.38900,-1.107000,2.783000,0.82230,-0.2244,-0.07886,...,-0.248300,0.02567,-0.712000,-0.61230,0.159300,4.08600,-0.24040,-0.27320,-0.45140,-2.23200
9,-0.173800,60.0,1.57800,-1.16500,-0.64160,0.642000,3.031000,0.09344,-1.2300,-0.16860,...,1.761000,3.21900,1.750000,-0.50240,-0.782000,1.70400,2.57200,0.90670,0.93300,-2.31600


In [12]:
display(p_df)

,0
0,5.0
1,3.0
2,1.0
3,1.0
4,2.0
...,...
225217,3.0
225218,2.0
225219,4.0
225220,2.0


In [4]:
import concurrent.futures
import importlib
from tqdm import tqdm
import inspect

def collect_function(calvins_utils=True):
    # Ask the user for the module and function name
    if calvins_utils:
        module_name = input("Please enter the name of the python file to use within the calvin_utils directory (dont include extension): ")
        module_name = 'calvin_utils.' + module_name
    else:
        module_name = input("Please enter the name of the module with target function in this way (dont include extension): <directory_name.python_file_name> ")
    function_name = input("Please enter the name of the target function to be run: ")

    # Feedback to user
    print('Checking for module: ', module_name)
    print('Checking in module for function: ', function_name)
    
    # Import the function
    try:
        module = importlib.import_module(module_name)
        function_to_run = getattr(module, function_name)
    except (ModuleNotFoundError, AttributeError) as e:
        print(f"Error: {e}")
    return function_to_run

def collect_args(func):
    # Get the function signature and parameters
    signature = inspect.signature(func)
    parameters = signature.parameters
 
    # Print the docstring if present
    if func.__doc__:
        print("Function documentation:")
        print(func.__doc__)
        print()

    # Collect user input based on the function parameters
    args = {}
    for name, param in parameters.items():
        # Check if the parameter has a default value
        if param.default == inspect.Parameter.empty:
            user_input = input(f"Please enter a value for '{name}': ")
        else:
            user_input = input(f"Please enter a value for '{name}' (default: {param.default}): ")
        
        # Convert the input to the correct type
        user_input = param.annotation(user_input) if param.annotation != inspect.Parameter.empty else user_input
        args[name] = user_input

    return args

In [5]:
func = collect_function()
args = collect_args(func)

In [10]:
print(func)
print(args)

<function permuted_patient_label_delta_r_map at 0x181eec3a0>
{'dataframe_to_permute': 'mx_1', 'observed_delta_r_map': 'mx_2', 'column_of_interest': "'age at DOS'", 'threshold_of_interest': '65', 'n_permutations': '1'}


In [9]:
importlib.import_module('calvin_utils.network_mapping_utilities')

<module 'calvin_utils.network_mapping_utilities' from '/Users/cu135/Library/CloudStorage/OneDrive-Personal/OneDrive_Documents/Work/PostDoc/Nimlab/software_env/python_modules/nimlab/calvin_utils/network_mapping_utilities.py'>